In [1]:
%matplotlib inline

In [2]:
import LFPy
import lfpykit
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# LFPy.Cell parameters
cellParameters = {
    'morphology': 'L5_Mainen96_LFPy.hoc',  # morphology file
    'v_init': -65,                         # initial voltage
    'cm': 1.0,                             # membrane capacitance
    'Ra': 150,                             # axial resistivity
    'passive': True,                       # insert passive channels
    'passive_parameters': {"g_pas": 1. / 3E4,
                           "e_pas": -65},  # passive params
    'dt': 2**-4,                           # simulation time res
    'nsegs_method': 'lambda_f',    # discretization rule
    'lambda_f': 1000               # frequency (Hz)
}

# create LFPy.Cell instance
cell = LFPy.Cell(**cellParameters)
cell.set_rotation(x=4.98919, y=-4.33261, z=0.)

print(f'total number of segments: {cell.totnsegs}')

# parameters for line source potential
el_params = dict(
    x = np.linspace(0, 1000, 1001),
    y = np.zeros(1001),
    z = np.zeros(1001),
    sigma = 0.3
)

cell.simulate(rec_imem=True)

total number of segments: 3078


In [4]:
# create line-source potential predictor
lsp = lfpykit.LineSourcePotential(cell, **el_params)

In [5]:
%%prun -s cumulative -q -l 20 -T prun0
for i in range(100):
    lsp.get_transformation_matrix()

 
*** Profile printout saved to text file 'prun0'.


In [6]:
print(open('prun0', 'r').read())

         2102403 function calls in 11.128 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   11.128   11.128 {built-in method builtins.exec}
        1    0.011    0.011   11.128   11.128 <string>:1(<module>)
      100    0.000    0.000   11.117    0.111 models.py:440(get_transformation_matrix)
      100    0.237    0.002   11.116    0.111 models.py:462(_get_transform)
   100100    0.187    0.000   10.880    0.000 lfpcalc.py:355(calc_lfp_linesource)
   100100    3.490    0.000   10.692    0.000 lfpcalc.py:402(_calc_lfp_linesource)
   100100    1.542    0.000    1.542    0.000 lfpcalc.py:560(_h_calc)
   100100    1.476    0.000    1.476    0.000 lfpcalc.py:520(_linesource_calc_case1)
   100100    1.294    0.000    1.294    0.000 lfpcalc.py:579(_r2_calc)
   100100    1.091    0.000    1.091    0.000 lfpcalc.py:547(_deltaS_calc)
   400400    0.084    0.000    0.813    0.000 <__array_function__ inter

In [7]:
class CyLSP(lfpykit.LineSourcePotential):
    '''test class using linesource extension'''
    def __init__(self, cell, x, y, z, sigma):
        super().__init__(cell, x, y, z, sigma)

    def get_transformation_matrix(self):
        '''
        Get linear response matrix

        Returns
        -------
        response_matrix: ndarray
            shape (n_coords, n_seg) ndarray

        Raises
        ------
        AttributeError
            if ``cell is None``
        '''
        if self.cell is None:
            raise AttributeError(
                '{}.cell is None'.format(self.__class__.__name__))
        if self.cell.d.ndim == 2:
            r_limit = self.cell.d.mean(axis=-1) / 2
        else:
            r_limit = self.cell.d / 2

        return lfpykit.linesource._get_transform(cell_x=self.cell.x,
                              cell_y=self.cell.y,
                              cell_z=self.cell.z,
                              x=self.x,
                              y=self.y,
                              z=self.z,
                              sigma=self.sigma,
                              r_limit=r_limit)

In [8]:
# create line-source potential predictor
cylsp = CyLSP(cell, **el_params)

In [9]:
%%prun -s cumulative -q -l 20 -T prun2
for i in range(100):
    cylsp.get_transformation_matrix()

 
*** Profile printout saved to text file 'prun2'.


In [10]:
print(open('prun2', 'r').read())

         1201403 function calls in 11.052 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   11.052   11.052 {built-in method builtins.exec}
        1    0.011    0.011   11.052   11.052 <string>:1(<module>)
      100    0.000    0.000   11.040    0.110 1624091361.py:6(get_transformation_matrix)
      100   10.238    0.102   11.040    0.110 {built-in method lfpykit.linesource._get_transform}
   400400    0.074    0.000    0.802    0.000 <__array_function__ internals>:2(where)
   400400    0.713    0.000    0.713    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   400400    0.016    0.000    0.016    0.000 multiarray.py:341(where)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


In [11]:
np.all(cylsp.get_transformation_matrix() == lsp.get_transformation_matrix())

True

In [12]:
# create point-source potential predictor
psp = lfpykit.PointSourcePotential(cell, **el_params)

In [13]:
%%prun -s cumulative -q -l 20 -T prun1
for i in range(100):
    psp.get_transformation_matrix()

 
*** Profile printout saved to text file 'prun1'.


In [14]:
print(open('prun1', 'r').read())

         3203403 function calls in 8.445 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    8.445    8.445 {built-in method builtins.exec}
        1    0.011    0.011    8.445    8.445 <string>:1(<module>)
      100    0.224    0.002    8.434    0.084 models.py:278(get_transformation_matrix)
   100100    1.271    0.000    8.211    0.000 lfpcalc.py:597(calc_lfp_pointsource)
   300300    0.072    0.000    6.606    0.000 {method 'mean' of 'numpy.ndarray' objects}
   300300    0.803    0.000    6.535    0.000 _methods.py:162(_mean)
   300300    5.459    0.000    5.459    0.000 {method 'reduce' of 'numpy.ufunc' objects}
   100100    0.333    0.000    0.333    0.000 lfpcalc.py:680(_check_rlimit_point)
   300300    0.193    0.000    0.227    0.000 _methods.py:66(_count_reduce_items)
   600600    0.025    0.000    0.025    0.000 {built-in method builtins.isinstance}
   600600    0.025    0.000    0.025